In [1]:
import pandas as pd
import plotly.express as px


In [2]:
FOLDER = 'data/generation_lab_survey/'
df_results = pd.read_excel(f'{FOLDER}raw_survey_1323.xlsx',index_col=[1],skiprows=[1])
df_questions = pd.read_excel(f'{FOLDER}raw_survey_1323.xlsx',index_col=[1],skiprows=[2]).head(1).T

In [3]:
uses_list = ['30 minutes to 1 hour','1 to 2 hours','2 to 3 hours','More than 3 hours']
doesnt_use_list = ['I do not use this platform']

agree_list = ['Strongly Agree','Moderately Agree','Slightly Agree']
disagree_list = ['Strongly Disagree','Moderately Disagree','Slightly Disagree']


In [4]:

question_dict = {'1.9_1': 'Agree','1.9_2':'Agree','1.9_3':'Agree','1.9_4':'Disagree','1.9_5':'Disagree','1.9_6': 'Disagree','1.9_7': 'Agree','1.9_8': 'Agree','1.9_9':'Agree','1.9_10':'Disagree','1.9_11':'Disagree','1.9_12':'Disagree'}
platform_dict = {"TikTok":'1.3_4',"Instagram":'1.3_1',"X":'1.3_3'}

for platform in platform_dict.keys():
    df_results[platform] = None
    mask = (df_results[platform_dict[platform]].isin(uses_list))     
    df_results.loc[mask,platform] = True

    mask = (df_results[platform_dict[platform]].isin(doesnt_use_list))     
    df_results.loc[mask,platform] = False 


In [5]:

df_answer_summary = pd.DataFrame(index=pd.MultiIndex.from_tuples([], names=["Platform","Uses","Question"]), columns=['Agree', 'Neutral', 'Disagree'])

for question in question_dict.keys(): 


    df_results[f'{question}_agree'] = None

    mask = df_results[question].isin(agree_list)
    df_results.loc[mask,f'{question}_agree'] = True

    mask = df_results[question].isin(disagree_list)
    df_results.loc[mask,f'{question}_agree'] = False

    for platform in platform_dict.keys():
        
        for uses_platform in [True,False]:
            
            if uses_platform:
                mask = (df_results[platform] == True)
            else:
                mask = (df_results[platform] == False)
            
            
            
            df_answer_summary.loc[(platform,uses_platform,question),'Agree'] = df_results.loc[mask,f'{question}_agree'].sum() 
            df_answer_summary.loc[(platform,uses_platform,question),'Neutral'] = df_results.loc[mask,f'{question}_agree'].isna().sum() 
            df_answer_summary.loc[(platform,uses_platform,question),'Disagree'] = mask.sum() - df_results.loc[mask,f'{question}_agree'].sum() - df_results.loc[mask,f'{question}_agree'].isna().sum() 



/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/3349402800.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_answer_summary.loc[(platform,uses_platform,question),'Neutral'] = df_results.loc[mask,f'{question}_agree'].isna().sum()
/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/3349402800.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_answer_summary.loc[(platform,uses_platform,question),'Disagree'] = mask.sum() - df_results.loc[mask,f'{question}_agree'].sum() - df_results.loc[mask,f'{question}_agree'].isna().sum()
/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/3349402800.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_answer_summary.loc[(platform,uses_platform,question),'Agree'] = df_results.loc[mask,f'{question}_agree'].sum()
/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/3349402800.py:25: PerformanceWarning: ind

In [6]:
for index, row in df_answer_summary.iterrows():
    total_responses = row['Agree'] + row['Neutral'] + row['Disagree']
    if total_responses > 0:
        df_answer_summary.loc[index, 'Agree %'] = round((row['Agree'] / total_responses)*100, 1)
        df_answer_summary.loc[index, 'Neutral %'] = round((row['Neutral'] / total_responses)*100, 1)
        df_answer_summary.loc[index, 'Disagree %'] = round((row['Disagree'] / total_responses)*100, 1)
 

for question in question_dict.keys(): 
    for platform in platform_dict.keys():
        for uses_platform in [True,False]:
            df_answer_summary.loc[(platform,uses_platform,question),'Pro_Israel_Perc'] = df_answer_summary.loc[(platform,uses_platform,question),f'{question_dict[question]} %']

/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/4066486530.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_answer_summary.loc[index, 'Agree %'] = round((row['Agree'] / total_responses)*100, 1)
/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/4066486530.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_answer_summary.loc[index, 'Neutral %'] = round((row['Neutral'] / total_responses)*100, 1)
/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/4066486530.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_answer_summary.loc[index, 'Disagree %'] = round((row['Disagree'] / total_responses)*100, 1)
/var/folders/8s/vl53fbd96rzd3wn49w05rygr0000gn/T/ipykernel_97854/4066486530.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_answer_summary.loc[index, 'Agree %'] = round((row['Agree'] / total_responses)*100, 1)
/var/folde

In [7]:
df_answer_summary_gb = df_answer_summary.groupby(['Platform','Uses'])['Pro_Israel_Perc'].mean()
df_answer_diff = pd.DataFrame(index=pd.MultiIndex.from_tuples([], names=["Platform"]), columns=['Difference'])
for platform in platform_dict.keys():
        df_answer_diff.loc[(platform),'Difference'] = (1-df_answer_summary_gb.loc[(platform,True)]/(df_answer_summary_gb.loc[(platform,False)]))


In [10]:

colors = ['#69C9D0', '#833AB4', 'black']  # Define your color sequence here

# Assign a different color to each row in the dataframe
df_answer_diff['Color'] = [colors[i % len(colors)] for i in range(len(df_answer_diff))]

# Creating a bar chart using Plotly with different colors for each bar
fig = px.bar(df_answer_diff.reset_index(), x='Platform', y='Difference', color='Color', color_discrete_map='identity')

fig.update_yaxes(tickformat='+.0%')
fig.update_xaxes(tickfont=dict(size=38),automargin=True)


# Updating layout for clarity
fig.update_layout(
    paper_bgcolor='white',  # Set the background color of the paper (outside plot)
    plot_bgcolor='white',   # Set the background color of the plot
    font=dict(size=22),
    title=dict(text='Relationship Between Platform Usage and Antisemitic/Anti-Israel Views', font=dict(size=40)),
    yaxis_title=dict(text='Change in likelihood (user/non-user)',font=dict(size=22)),
    xaxis_title='',  # Remove the x-axis label
    width=1800, height=900,
    margin=dict(t=150, b=200),
)

fig.add_annotation(
    text="Source: Survey of 1,323 Americans aged under 30 years old conducted by Generation Lab. Includes respondents from 47 States.",  # Replace with your source
    xref="paper", x=0.0,  # x=0.5 for center alignment, x=1 for right alignment
    yref="paper", y=-0.2,  # Negative value to place it below the x-axis
    showarrow=False,
    font=dict(size=18,color="grey"),  # You can adjust the font size and color
    align="left", # or "right" for right alignment
)
fig.add_annotation(
    text="Compares people who use a platform for more than 30 minutes a day with those who don't use it at all. ",  # Replace with your source
    xref="paper", x=0.0,  # x=0.5 for center alignment, x=1 for right alignment
    yref="paper", y=-0.25,  # Negative value to place it below the x-axis
    showarrow=False,
    font=dict(size=18,color="grey"),  # You can adjust the font size and color
    align="left", # or "right" for right alignment
)
fig.add_annotation(
    text="Raw survey asked about use of Twitter/Threads. Since X/Twitter has 10x the usage of Threads we assume people are mostly referring to X/Twitter usage.",  # Replace with your source
    xref="paper", x=0.0,  # x=0.5 for center alignment, x=1 for right alignment
    yref="paper", y=-0.3,  # Negative value to place it below the x-axis
    showarrow=False,
    font=dict(size=18,color="grey"),  # You can adjust the font size and color
    align="left", # or "right" for right alignment
)

# Displaying the figure
fig.show()

In [9]:
df_answer_summary.loc[df_answer_summary.index.get_level_values(2).isin(['1.9_1','1.9_2','1.9_3','1.9_4','1.9_5','1.9_6']),'Focus'] = 'Anti-Semitism'
df_answer_summary.loc[df_answer_summary.index.get_level_values(2).isin(['1.9_7','1.9_8','1.9_9','1.9_10','1.9_11','1.9_12']),'Focus'] = 'Anti-Israel'

df_answer_summary_plot = df_answer_summary.reset_index()

df_answer_summary_gb = df_answer_summary_plot.groupby(['Platform','Uses','Focus'])['Pro_Israel_Perc'].mean()
df_answer_diff_with_focus= pd.DataFrame(index=pd.MultiIndex.from_tuples([], names=["Platform","Focus"]), columns=['Difference'])
platform = 'TikTok'
for focus in ['Anti-Semitism','Anti-Israel']:
    df_answer_diff_with_focus.loc[(platform,focus),'Difference'] = (1-df_answer_summary_gb.loc[(platform,True,focus)]/(df_answer_summary_gb.loc[(platform,False,focus)]))*100 

df_answer_diff_with_focus

Difference
Platform Focus                   
TikTok   Anti-Semitism  20.475228
         Anti-Israel    12.285491

In [11]:
df_answer_summary_dump = df_answer_summary.reset_index().merge(df_questions,left_on='Question',right_index=True).rename(columns={'Response ID':'Question Text'})
df_answer_summary_dump['Question Text'] = df_answer_summary_dump['Question Text'].str[80:]

In [12]:
df_answer_summary_dump.to_csv(f'{FOLDER}survey_israel_questions_clean.csv')

In [13]:
count = 1
for question in df_answer_summary_dump['Question Text'].drop_duplicates():
    print(f"{count}. {question}")
    count += 1

1. Jewish people can be trusted just as much as other American people in business.
2. Jewish people are just as loyal to America as other American people.
3. I am just as open to having Jewish friends as I am to having friends from other sections of American society.
4. Compared to other groups, Jewish people have too much power in the media.
5. Jewish people talk about the Holocaust just to further their political agenda.
6. Jewish people chase money more than other people do.
7. I am comfortable spending time with people who openly support Israel.
8. Israel has a right to exist as a homeland for the Jewish people.
9. Israel is right to defend itself against those who want to destroy it.
10. Israel and its supporters are a bad influence on our democracy.
11. Israel can get away with anything because its supporters control the media.
12. Israel treats the Palestinians like the Nazis treated the Jews.


In [ ]:
df_answer_summary_dump